In [1]:
from unsloth import FastLanguageModel
import torch
import json

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ubuntu/miniforge3/envs/AlphaBuffet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [23]:
hf_key = "***"
repo = "AlphaBuffett"

In [4]:
models = [
    "unsloth/Mistral-Small-24B-Base-2501",
    "unsloth/Mistral-Small-24B-Base-2501-bnb-4bit",
    "unsloth/Mistral-Small-24B-Base-2501-unsloth-bnb-4bit",
    "unsloth/Mistral-Small-24B-Instruct-2501",
    "unsloth/Mistral-Small-24B-Instruct-2501-bnb-4bit",
    "unsloth/Mistral-Small-24B-Instruct-2501-unsloth-bnb-4bit",
    "unsloth/phi-4",
    "unsloth/phi-4-bnb-4bit",
    "unsloth/phi-4-unsloth-bnb-4bit",
] 

dtype = None
load_in_4bit = False
max_seq_length = 8192

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = models[0],
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.12: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA H100 PCIe. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 9.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 10/10 [00:17<00:00,  1.79s/it]


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.12 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [6]:
from unsloth.chat_templates import get_chat_template
"""
#Phi
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)

"""
# Mistral
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True,
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize = False, add_generation_prompt = False
        )
        for convo in convos
    ]
    return { "text" : texts, }
pass

Unsloth: Will map <|im_end|> to EOS = </s>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [9]:
from datasets import load_dataset
dataset = load_dataset(
    "json",
    data_files = "/home/ubuntu/AlphaBuffett/Dataset/Processed/Ground Truth/dataset_combined.json",
    split = "train",
)

Generating train split: 7265 examples [00:00, 46406.14 examples/s]


In [10]:
dataset[5]['conversations']

[{'from': 'human',
  'value': 'How do you approach the challenge of maintaining discipline when market conditions soften and competitors start offering unrealistic rates?'},
 {'from': 'gpt',
  'value': "This is one of the most difficult challenges in our insurance operations, and it requires what I call unusual managerial discipline. You see, when markets loosen and rates become inadequate, the natural institutional behavior is to fight to keep your business. Nobody likes to see the other fellow take away their customers. But we've learned that you sometimes have to philosophically accept reduced volume. Look at what happened with National Indemnity - we've had excellent underwriting margins and large volume gains when many competitors contracted or withdrew after the 1974-75 crisis. But these conditions won't last forever. When the market softens, we have to be willing to let business go when prices become foolish. It's like being the designated driver at a party - you might not be ha

In [11]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(dataset)

Standardizing format: 100%|██████████| 7265/7265 [00:00<00:00, 35822.20 examples/s]


In [12]:
dataset[5]['conversations']

[{'content': 'How do you approach the challenge of maintaining discipline when market conditions soften and competitors start offering unrealistic rates?',
  'role': 'user'},
 {'content': "This is one of the most difficult challenges in our insurance operations, and it requires what I call unusual managerial discipline. You see, when markets loosen and rates become inadequate, the natural institutional behavior is to fight to keep your business. Nobody likes to see the other fellow take away their customers. But we've learned that you sometimes have to philosophically accept reduced volume. Look at what happened with National Indemnity - we've had excellent underwriting margins and large volume gains when many competitors contracted or withdrew after the 1974-75 crisis. But these conditions won't last forever. When the market softens, we have to be willing to let business go when prices become foolish. It's like being the designated driver at a party - you might not be having as much f

In [13]:
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map: 100%|██████████| 7265/7265 [00:00<00:00, 17596.69 examples/s]


In [14]:
dataset[5]['text']

"<|im_start|>user\nHow do you approach the challenge of maintaining discipline when market conditions soften and competitors start offering unrealistic rates?<|im_end|>\n<|im_start|>assistant\nThis is one of the most difficult challenges in our insurance operations, and it requires what I call unusual managerial discipline. You see, when markets loosen and rates become inadequate, the natural institutional behavior is to fight to keep your business. Nobody likes to see the other fellow take away their customers. But we've learned that you sometimes have to philosophically accept reduced volume. Look at what happened with National Indemnity - we've had excellent underwriting margins and large volume gains when many competitors contracted or withdrew after the 1974-75 crisis. But these conditions won't last forever. When the market softens, we have to be willing to let business go when prices become foolish. It's like being the designated driver at a party - you might not be having as mu

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Applying chat template to train dataset (num_proc=2): 100%|██████████| 7265/7265 [00:01<00:00, 4552.24 examples/s]
Tokenizing train dataset (num_proc=2): 100%|██████████| 7265/7265 [00:01<00:00, 4327.09 examples/s]


In [16]:
from unsloth.chat_templates import train_on_responses_only
"""
#Phi

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user<|im_sep|>",
    response_part="<|im_start|>assistant<|im_sep|>",
)

"""
# Mistral

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user\n",
    response_part="<|im_start|>assistant\n",
)


Map: 100%|██████████| 7265/7265 [00:01<00:00, 4453.59 examples/s]


In [17]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<s><|im_start|>user\nHow do you approach the challenge of maintaining discipline when market conditions soften and competitors start offering unrealistic rates?<|im_end|>\n<|im_start|>assistant\nThis is one of the most difficult challenges in our insurance operations, and it requires what I call unusual managerial discipline. You see, when markets loosen and rates become inadequate, the natural institutional behavior is to fight to keep your business. Nobody likes to see the other fellow take away their customers. But we've learned that you sometimes have to philosophically accept reduced volume. Look at what happened with National Indemnity - we've had excellent underwriting margins and large volume gains when many competitors contracted or withdrew after the 1974-75 crisis. But these conditions won't last forever. When the market softens, we have to be willing to let business go when prices become foolish. It's like being the designated driver at a party - you might not be having as

In [18]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                         \nThis is one of the most difficult challenges in our insurance operations, and it requires what I call unusual managerial discipline. You see, when markets loosen and rates become inadequate, the natural institutional behavior is to fight to keep your business. Nobody likes to see the other fellow take away their customers. But we've learned that you sometimes have to philosophically accept reduced volume. Look at what happened with National Indemnity - we've had excellent underwriting margins and large volume gains when many competitors contracted or withdrew after the 1974-75 crisis. But these conditions won't last forever. When the market softens, we have to be willing to let business go when prices become foolish. It's like being the designated driver at a party - you might not be having as much fun as everyone else, but you're making the right long-term decision. The key is maintaining that discipline even when it runs counter to what eve

In [19]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H100 PCIe. Max memory = 79.097 GB.
44.617 GB of memory reserved.


In [20]:
import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,265 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 2,724
 "-____-"     Number of trainable parameters = 184,811,520


Step,Training Loss
1,1.995400
2,1.989700
3,1.968000
4,1.903700
5,1.867200
6,1.685700
7,1.820200
8,1.729800
9,1.808000
10,1.601300


In [21]:
from huggingface_hub import delete_repo

delete_repo(
    repo_id=repo,
    token=hf_key,
    missing_ok = True
)

In [24]:
# save_method = "merged_4bit" or "merged_16bit"

model.push_to_hub_merged(repo, tokenizer, save_method = "merged_16bit", commit_message = "Initial Commit", commit_description = "Initial Commit after Lora Fine Tuning", token = hf_key, private = False)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 153.17 out of 221.18 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 40/40 [01:34<00:00,  2.36s/it]


Unsloth: Saving to organization with address brokenlander/AlphaBuffett
 Done.h: Saving tokenizer...
Unsloth: Saving to organization with address brokenlander/AlphaBuffett
Unsloth: Uploading all files... Please wait...


100%|██████████| 11/11 [01:10<00:00,  6.45s/it]


Done.
Saved merged model to https://huggingface.co/None/AlphaBuffett
